# Install necessary libraries

Using PyTorch here because huggingface models and facebook 😛

In [1]:
%pip install -q torch transformers datasets huggingface_hub accelerate evaluate rouge_score textstat

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 21.2 MB/s eta 0:00:00
  

In [2]:
%pip install -U -q datasets

In [3]:
# import stuff
import pandas as pd
import torch
from datasets import load_dataset, DatasetDict, Dataset
from transformers import DataCollatorForSeq2Seq
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import TrainingArguments, Trainer
import evaluate
import textstat

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train on CNN Dataset

i want to do this so the model learns to summarize first. because fine-tuning bart-large-cnn is kinda expensive

In [5]:
# load cnn dataset
cnn = load_dataset("abisee/cnn_dailymail", "3.0.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
# load models
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [7]:
# a preprocess function
def preprocess_cnn(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=512,
        truncation=True,
        padding=False  # Let DataCollator handle padding
    )


    labels = tokenizer(
        examples["highlights"],
        max_length=128,
        truncation=True,
        padding=False
    )

    labels["input_ids"] = [
        [label if label != tokenizer.pad_token_id else -100 for label in seq]
        for seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [9]:
tokenized_cnn = cnn["train"].select(range(10000))  # use 10k samples for fine-tuning
tokenized_cnn = tokenized_cnn.map(preprocess_cnn, batched=True, remove_columns=["article", "highlights", "id"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./cnn_bart_base",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    eval_strategy="no",  # skip eval for this warm-up stage
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    fp16=True,  # if gpu supports
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_cnn,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-10-136420024.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
100,2.836100
200,2.585300
300,2.469900
400,2.501100
500,2.424500
600,2.421900
700,2.460500
800,2.357400
900,2.292200
1000,2.384400


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=7500, training_loss=1.934480977376302, metrics={'train_runtime': 1914.1986, 'train_samples_per_second': 15.672, 'train_steps_per_second': 3.918, 'total_flos': 9137198549606400.0, 'train_loss': 1.934480977376302, 'epoch': 3.0})

# Load the dataset

In [11]:
dataset = load_dataset("json", data_files="data.jsonl")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
# shuffling first
dataset = dataset.shuffle(seed=42)

# 78% train, 11% validation, 11% test
train_testvalid = dataset.train_test_split(test_size=0.22, seed=42)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5, seed=42)

# Final splits
dataset_splits = {
    "train": train_testvalid["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
}

In [13]:
# save to file (idc)
dataset_splits["train"].to_json("train.jsonl")
dataset_splits["validation"].to_json("val.jsonl")
dataset_splits["test"].to_json("test.jsonl")

Creating json from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

956958

In [14]:
# ok load again
data_files = {
    "train": "train.jsonl",
    "validation": "val.jsonl",
    "test": "test.jsonl"
}

dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Tokenization

In [15]:
def preprocess_function(examples):
    # tokenizing
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=512,
        truncation=True,
        padding=False
    )

    # tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=128,
            truncation=True,
            padding=False
        )

    # replace padding token id's in labels with -100
    labels["input_ids"] = [
        [token if token != tokenizer.pad_token_id else -100 for token in label_seq]
        for label_seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [16]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["input_text", "target_text"])

Map:   0%|          | 0/7693 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7693
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1085
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1085
    })
})

# Data Collator

In [18]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

## Training

In [19]:
training_args = TrainingArguments(
    output_dir="./wiki_bart_finetuned",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,  # reduced from 10
    learning_rate=2e-5,  # lower LR for stability
    weight_decay=0.01,   # regularization
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    save_total_limit=2,
    fp16=True,  # if supported
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

In [20]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,  # still ok for now
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

/tmp/ipython-input-20-2536265044.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.363200,1.205572
2,1.189100,1.174452
3,1.121400,1.166353


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5772, training_loss=1.2563071736674793, metrics={'train_runtime': 846.0785, 'train_samples_per_second': 27.278, 'train_steps_per_second': 6.822, 'total_flos': 3120231649167360.0, 'train_loss': 1.2563071736674793, 'epoch': 3.0})

In [26]:
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

# Evaluating

In [30]:
import torch
from torch.utils.data import DataLoader

def evaluate_model(model, dataset, tokenizer, batch_size=8, max_input_length=512, max_summary_length=128):
    model.eval()
    device = model.device

    dataloader = DataLoader(dataset, batch_size=batch_size)
    predictions = []
    references = []

    for batch in dataloader:
        # Assuming dataset returns dicts with raw text fields
        input_texts = batch["input_text"]
        ref_texts = batch["target_text"]

        # Tokenize batch inputs
        inputs = tokenizer(
            input_texts,
            max_length=max_input_length,
            padding="longest",
            truncation=True,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            generated_ids = model.generate(
                inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_summary_length,
                num_beams=4,
                early_stopping=True,
            )

        batch_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        predictions.extend(batch_preds)
        references.extend(ref_texts)

    # Compute ROUGE
    rouge = evaluate.load("rouge")
    rouge_results = rouge.compute(predictions=predictions, references=references, use_stemmer=True)

    # Compute Flesch Reading Ease (FRE) scores
    pred_fre = [textstat.flesch_reading_ease(p) for p in predictions]
    ref_fre = [textstat.flesch_reading_ease(r) for r in references]
    avg_pred_fre = sum(pred_fre) / len(pred_fre)
    avg_ref_fre = sum(ref_fre) / len(ref_fre)

    print("ROUGE scores:")
    for k in ['rouge1', 'rouge2', 'rougeL']:
        print(f"  {k}: {rouge_results[k]:.4f}")

    print(f"\nAverage Flesch Reading Ease (Predictions): {avg_pred_fre:.2f}")
    print(f"Average Flesch Reading Ease (References): {avg_ref_fre:.2f}")

    return rouge_results, avg_pred_fre, avg_ref_fre


# Usage example:
rouge_scores, pred_fre, ref_fre = evaluate_model(model, dataset["test"], tokenizer)


ROUGE scores:
  rouge1: 0.6498
  rouge2: 0.5128
  rougeL: 0.5951

Average Flesch Reading Ease (Predictions): 58.29
Average Flesch Reading Ease (References): 63.37


In [31]:
trainer.save_model("./wiki_bart_finetuned")
tokenizer.save_pretrained("./wiki_bart_finetuned")

('./wiki_bart_finetuned/tokenizer_config.json',
 './wiki_bart_finetuned/special_tokens_map.json',
 './wiki_bart_finetuned/vocab.json',
 './wiki_bart_finetuned/merges.txt',
 './wiki_bart_finetuned/added_tokens.json')

In [32]:
!cp -r /content/wiki_bart_finetuned /content/drive/MyDrive/

generate some summaries

In [58]:
model_path = "/content/drive/MyDrive/wiki_bart_finetuned"
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

def summarize_text(text, model=model, tokenizer=tokenizer, max_length=512):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # move all inputs to model.device

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=6,
            early_stopping=False,
            length_penalty=1.2,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=2.5,
            no_repeat_ngram_size=3,
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [51]:
import textwrap

def pretty_print(*args, width=80, sep=' ', end='\n'):
    combined_text = sep.join(str(arg) for arg in args)
    print(textwrap.fill(combined_text, width=width), end=end)

In [60]:
input_text = """
One of the primary drivers of economic disparity is financialization -- the increasing dominance of financial markets, institutions, and elites over economic policy.
This shift has redirected wealth accumulation toward asset-holders while stagnating wage growth for lower-income populations.
Additionally, automation and artificial intelligence threaten to displace millions of low-skilled workers, further widening the income gap.
While proponents argue that technological progress fosters new industries and employment opportunities, critics highlight that the benefits disproportionately favor those with advanced education and capital investments.
"""

summary = summarize_text(input_text, model, tokenizer)
pretty_print("Original Text: ", input_text)
pretty_print("Generated Summary: ", summary)

Original Text:   One of the primary drivers of economic disparity is
financialization -- the increasing dominance of financial markets, institutions,
and elites over economic policy. This shift has redirected wealth accumulation
toward asset-holders while stagnating wage growth for lower-income populations.
Additionally, automation and artificial intelligence threaten to displace
millions of low-skilled workers, further widening the income gap. While
proponents argue that technological progress fosters new industries and
employment opportunities, critics highlight that the benefits disproportionately
favor those with advanced education and capital investments.
Generated Summary:  One of the primary drivers of economic disparity is
financialization, the increasing dominance of financial markets, institutions,
and elites over economic policy. This shift has redirected wealth accumulation
toward asset-holders while stagnating wage growth for lower-income populations.
Automation and artifi

In [61]:
input_text = """Neuroplasticity, also known as brain plasticity, refers to the brain's remarkable ability to reorganize itself by forming new neural connections throughout life. This ability enables the brain to adjust in response to new experiences, learning, injury, or environmental changes. For a long time, scientists believed that the brain's structure and function were relatively fixed after early childhood. However, modern neuroscience has shown that the brain remains adaptable well into adulthood and even old age.

When a person learns a new skill, such as playing a musical instrument or speaking a foreign language, the brain creates and strengthens neural pathways related to those tasks. Similarly, when part of the brain is damaged—such as after a stroke—other regions can sometimes take over the lost functions through rewiring. This has important implications for rehabilitation and cognitive therapy, providing hope that patients can recover abilities previously thought lost.

Neuroplasticity is also influenced by lifestyle factors. Regular physical exercise, quality sleep, engaging in challenging cognitive activities, and maintaining social connections have all been shown to promote brain plasticity. On the other hand, chronic stress, poor nutrition, and lack of mental stimulation can hinder the brain’s adaptive capacity.

Understanding and harnessing neuroplasticity is at the forefront of many educational and clinical strategies. Teachers, therapists, and neuroscientists are increasingly designing interventions that take advantage of the brain’s natural ability to change, aiming to improve learning outcomes, mental health, and recovery from neurological conditions."""

summary = summarize_text(input_text, model, tokenizer)
pretty_print("Original Text: ", input_text)
print("\n\n")
pretty_print("Generated Summary: ", summary)

Original Text:  Neuroplasticity, also known as brain plasticity, refers to the
brain's remarkable ability to reorganize itself by forming new neural
connections throughout life. This ability enables the brain to adjust in
response to new experiences, learning, injury, or environmental changes. For a
long time, scientists believed that the brain's structure and function were
relatively fixed after early childhood. However, modern neuroscience has shown
that the brain remains adaptable well into adulthood and even old age.  When a
person learns a new skill, such as playing a musical instrument or speaking a
foreign language, the brain creates and strengthens neural pathways related to
those tasks. Similarly, when part of the brain is damaged—such as after a
stroke—other regions can sometimes take over the lost functions through
rewiring. This has important implications for rehabilitation and cognitive
therapy, providing hope that patients can recover abilities previously thought
lost.  N

In [62]:
input_text = """The Ship of Theseus, also known as Theseus's Paradox, is a paradox and common thought experiment about whether an object is the same object after having all of its original components replaced over time, typically one after the other.

In Greek mythology, Theseus, the mythical king of the city of Athens, rescued the children of Athens from King Minos after slaying the minotaur and then escaped onto a ship going to Delos. Each year, the Athenians would commemorate this by taking the ship on a pilgrimage to Delos to honour Apollo. A question was raised by ancient philosophers: If no pieces of the original made up the current ship, was it still the Ship of Theseus? Furthermore, if it was no longer the same, when had it ceased existing as the original ship? Thomas Hobbes raised the further question of how to consider a second ship that had been built entirely from pieces removed from the original.

In contemporary philosophy, the thought experiment has applications to the philosophical study of identity over time. Within the contemporary philosophy of mind, it has inspired a variety of proposed solutions and concepts regarding the persistence of personal identity."""

summary = summarize_text(input_text, model, tokenizer)
pretty_print("Original Text:", input_text)
print()
pretty_print("Generated Summary:", summary)

Original Text: The Ship of Theseus, also known as Theseus's Paradox, is a
paradox and common thought experiment about whether an object is the same object
after having all of its original components replaced over time, typically one
after the other.  In Greek mythology, Theseus, the mythical king of the city of
Athens, rescued the children of Athens from King Minos after slaying the
minotaur and then escaped onto a ship going to Delos. Each year, the Athenians
would commemorate this by taking the ship on a pilgrimage to Delos to honour
Apollo. A question was raised by ancient philosophers: If no pieces of the
original made up the current ship, was it still the Ship of Theseus?
Furthermore, if it was no longer the same, when had it ceased existing as the
original ship? Thomas Hobbes raised the further question of how to consider a
second ship that had been built entirely from pieces removed from the original.
In contemporary philosophy, the thought experiment has applications to the
phi

In [59]:
input_text = """Etymology (/ˌɛtɪˈmɒlədʒi/ ET-im-OL-ə-jee) is the study of the origin and evolution of words—including their constituent units of sound and meaning—across time. In the 21st century a subfield within linguistics, etymology has become a more rigorously scientific study. Most directly tied to historical linguistics, philology, and semiotics, it additionally draws upon comparative semantics, morphology, pragmatics, and phonetics in order to attempt a comprehensive and chronological catalogue of all meanings and changes that a word (and its related parts) carries throughout its history. The origin of any particular word is also known as its etymology.
For languages with a long written history, etymologists make use of texts, particularly texts about the language itself, to gather knowledge about how words were used during earlier periods, how they developed in meaning and form, or when and how they entered the language. Etymologists also apply the methods of comparative linguistics to reconstruct information about forms that are too old for any direct information to be available. By analyzing related languages with a technique known as the comparative method, linguists can make inferences about their shared parent language and its vocabulary. In this way, word roots in many European languages, for example, can be traced back to the origin of the Indo-European language family.
Even though etymological research originated from the philological tradition, much current etymological research is done on language families where little or no early documentation is available, such as Uralic and Austronesian.
"""

summary = summarize_text(input_text, model, tokenizer)
pretty_print("Original Text:", input_text)
print()
pretty_print("Generated Summary:", summary)

Original Text: Etymology (/ˌɛtɪˈmɒlədʒi/ ET-im-OL-ə-jee) is the study of the
origin and evolution of words—including their constituent units of sound and
meaning—across time. In the 21st century a subfield within linguistics,
etymology has become a more rigorously scientific study. Most directly tied to
historical linguistics, philology, and semiotics, it additionally draws upon
comparative semantics, morphology, pragmatics, and phonetics in order to attempt
a comprehensive and chronological catalogue of all meanings and changes that a
word (and its related parts) carries throughout its history. The origin of any
particular word is also known as its etymology. For languages with a long
written history, etymologists make use of texts, particularly texts about the
language itself, to gather knowledge about how words were used during earlier
periods, how they developed in meaning and form, or when and how they entered
the language. Etymologists also apply the methods of comparative lingui